# Problema de asignación
* Juan José de Jesús Hernández Beltrán - A00836747
* Kevin Jesús Martínez Trinidad - A00834493
* Miranda Isabel Rada Chau -A01285243
* Fedra Fernanda Mandujano López - A00835797
* Juan Marco Castro Trinidad - A01742821

In [ ]:
"""

"""
import time
import random
import numpy as np
random.seed(20)


""" -----------------------------------------------------------
                DEFINIR PARÁMETROS DEL PROBLEMA
----------------------------------------------------------- """

n = 9
capacidad = 1
costos = [10,9,5,9,8,3,6,4,7]


""" -----------------------------------------------------------
            DEFINIR CLASES DEL ALGORITMO GENÉTICO

Lista de parámetros a considerar...
(a) Tipo de cromosoma:
(b) Longitud:
(c) Criterio de inicialización:
(d) Criterio de infactibilidad:
(e) Criterio de paro:
(f) Función fitnes:
(g) Criterio de selección:
(h) Tamaño de la población:
(i) Probabilidad de cruce:
(j) Puntos de cruce:
(k) Lugar de cruce:
(l) Probabilidad de mutación:
(m) Criterio de reemplazo:
----------------------------------------------------------- """


class Cromosoma(object):
    """
    Inicializar el objeto aleatoriamente.
    """
    def __init__(self, genes_, longitud_):
        self.genes = genes_
        self.longitud = longitud_
        # Lista con los genes que conforman el cromosoma elegidos al azar.
        self.genotipo = [random.choice(self.genes) for i in range(self.longitud)]


    """
    Configura el genotipo del objeto como producto de un CRUCE.
    - padreA: Instancia Cromosoma correspondiente a un cromosoma padre
    - padreB: Instancia Cromosoma correspondiente a otro cromosoma padre
    - puntoCruce: Entero correspondiente al punto de cruce.
    - config: Qué configuración de cruce se utiliza...
        •) 0: Padre A + Padre B
        •) 1: Padre B + Padre A
    """
    def cruce(self, puntoDeCruce:int, config:int = 0, padreA = None, padreB = None):
        a = padreA.genotipo
        b = padreB.genotipo
        if config == 0:
            hijo = a[0 : puntoDeCruce] + b[puntoDeCruce:]
        else:
            hijo = b[0 : puntoDeCruce] + a[puntoDeCruce:]
        self.genotipo = hijo


    """
    Mostrar en la consola el genotipo del cromosoma.
    """
    def str(self):
        return str(self.genotipo) + ' Fitness: ' + str(self.funcionFitness())

    """
    Mutar los genes del cromosoma bajo cierta probabilidad.
    - probMutacion (opcional): Probabilidad independiente de que cada uno de los genes mute.
    Si el gen muta, se elige aleatoriamente entre los posibles valores de genes distintos al actual.
    """
    def mutacion(self, probMutacion = 0.1):
        for i in range(self.longitud):
            prob = random.random()
            if prob <= probMutacion:
                opciones = self.genes.copy()
                opciones.remove(self.genotipo[i])
                self.genotipo[i] = random.choice(opciones)


    """
    Comprobar el criterio de factibilidad sobre sí mismo.
    > Retorna: Bool que es True si es factible y False si no lo es.
    """
    def factible(self):
        # Calcular la suma de los pesos en la mochila
        aux1 = sum([self.genotipo[i] for i in range(0,3)])
        aux2 = sum([self.genotipo[i] for i in range(3,6)])
        aux3 = sum([self.genotipo[i] for i in range(6,9)])
        aux4 = sum([self.genotipo[i] for i in [0,3,6]])
        aux5 = sum([self.genotipo[i] for i in [1,4,7]])
        aux6 = sum([self.genotipo[i] for i in [2,5,8]])
        self.costo = aux1+aux2+aux3
        # Comparar con la capacidad de la mochila
        if aux1 == capacidad and aux2 == capacidad and aux3 == capacidad and aux4 == capacidad and aux5 == capacidad and aux6 == capacidad:
            return True
        else:
            return False

    """
    Evalúa la función fitness para un cromosoma.
    > Retorna: Float que corresponde a la evaluación del fitness de un cromosoma.
    Fitness = Función objetivo - 5 * max(0, peso_de_la_mochila - capacidad)
    Penaliza las soluciones infactibles.
    """
    def funcionFitness(self):
        # Calcular la suma de los beneficios en la mochila
        aux = (sum([self.genotipo[i] * costos[i] for i in range(self.longitud)]))
        # Penaliza las soluciones infactibles restandoles 5 * qué tanto se pasan.
        if not self.factible():
            aux += 100 * (self.costo - capacidad)
        aux = max(0, aux)
        return aux





class Genetico(object):

    """
    Inicialización.
    - genes: Lista que contiene el "alfabeto" de genes que pueden aparecer en los cromosomas.
    - longitud: Entero que corresponde a la longitud de los cromosomas.
    - tamanoPoblacion: Entero que corresponde al tamaño de la población a utilizar.
    - probCruce: Float entre 0 y 1 que representa la probabilidad de que dados dos posibles padres, estos se crucen.
    - puntoCruce: Entero que representa el punto de cruce (indexado desde 1).
    - probMutacion: Float entre 0 y 1 que representa la probabilidad de que un hijo mute.
    """
    def __init__(self, _genes, _longitud, _tamanoPoblacion, _probCruce, _probMutacion, _puntoCruce):
        # Asignar parámetros a variables locales
        self.genes = _genes
        self.longitud = _longitud
        self.tamanoPoblacion = _tamanoPoblacion
        self.probCruce = _probCruce
        self.probMutacion = _probMutacion
        self.puntoCruce = _puntoCruce

        # - generacion: Entero que corresponde al número de generación en el algoritmo
        self.generacion = 1

        # Inicializar población aleatoriamente
        self.poblacion = []
        while len(self.poblacion) < self.tamanoPoblacion:
            aux = Cromosoma(genes_ = self.genes, longitud_ = self.longitud)
            # Solo lo agrega a la población si es factible
            if aux.factible():
                self.poblacion.append(aux)
        print('>> Algoritmo inicialiado.')


    """
    Mostrar en la consola la población actual del algoritmo.
    """
    def show(self):
        print(f'>> Población actual (generación {self.generacion})')
        for i in self.poblacion:
            print(i.str())

    """
    Función principal.
    > Retorna: Instancia Cromosoma que corresponde al cromosoma de la solución determinada por el algoritmo.
    Ejecuta iteraciones del algoritmo hasta satisfacer el criterio de paro.
    """
    def run(self):
        print('>> Ejecutando iteraciones...\n•', end='')
        # Verificar criterio de paro
        while not self.criterioParo():
            # Sustituir la población actual por una nueva generación que desciende de ella
            self.nuevaGeneracion()
            self.generacion += 1
            print('•', end = '')
            if self.generacion % 100 == 0:
                print('')
        print('')

        # Si se cumple el criterio de paro...
        # Retornar el cromosoma con mejor fitness
        best = min(self.poblacion, key = lambda x: x.funcionFitness())
        print('>> Ejecución terminada.')
        print('>> Factibilidad:', best.factible())
        print('>> Mejor solución:', best.str())
        return best


    """
    Sustituye la población actual por una nueva generación que desciende de ella.
    Solo incluye en la población hijos factibles antes de mutar.
    """
    def nuevaGeneracion(self):
        # Elegir padres y cruzarlos hasta obtener una nueva población del mismo tamaño que la anterior
        nuevaPoblacion = []

        # Precalcular la distribución de probabilidad
        fits = [i.funcionFitness() for i in self.poblacion]
        suma = sum(fits)
        self.distProbabilidad = [i/suma for i in fits] # Se guarda la distribución
        self.distProbabilidad[-1] += 0.1 # Para asegurar que las probabilidades suman al menos 1

        while len(nuevaPoblacion) < self.tamanoPoblacion:
            # Elegir a los padres
            a, b = self.criterioSeleccion()
            # Generar la probabilidad de cruce
            prob = random.random()
            if prob <= self.probCruce:
                # Probar si el primer hijo es factible, si sí, se agrega a la población
                aux = Cromosoma(self.genes, self.longitud)
                aux.cruce(self.puntoCruce, 0, a, b)
                if aux.factible():
                    # Se agrega a la nueva población una versión mutada de aux
                    aux.mutacion(self.probMutacion)
                    nuevaPoblacion.append(aux)

                # Probar si el segundo hijo es factible, si sí, se agrega a la población
                aux = Cromosoma(self.genes, self.longitud)
                aux.cruce(self.puntoCruce, 1, a, b)
                if aux.factible():
                    # Se agrega a la nueva población una versión mutada de aux
                    aux.mutacion(self.probMutacion)
                    nuevaPoblacion.append(aux)

        # Conservar la nueva población
        self.poblacion = nuevaPoblacion


    """
    Comprobar si se cumple el criterio de paro.
    > Retorna: Bool que es verdadero si se cumple el criterio de paro.
    """
    def criterioParo(self):
        #if self.generacion <= 65:
        #    return False
        #else:
        #   return True

        best = min(self.poblacion, key = lambda x: x.funcionFitness())
        if best.funcionFitness() < 18 and best.factible()==True:
           return True
        else:
           return False

    """
    Determina que cromosomas serán seleccionados para procrear.
    > Retorna: Dos instancias de Cromosoma correspondientes a los cromosomas seleccionados de la población actual.
    Selección de ruleta, los mejores tienen más posibilidades de ser seleccionados.
    """
    def criterioSeleccion(self):
        # Elegir un padre
        sel = random.random()
        accum = 0
        a = self.poblacion[-1]
        for i in range(self.tamanoPoblacion):
            accum += self.distProbabilidad[i]
            if sel <= accum:
                a = self.poblacion[i]
                break

        # Elegir otro padre
        sel = random.random()
        accum = 0
        b = self.poblacion[-1]
        for i in range(self.tamanoPoblacion):
            accum += self.distProbabilidad[i]
            if sel <= accum:
                b = self.poblacion[i]
                break

        return a, b






""" -----------------------------------------------------------
                EJECUTA EL ALGORITMO GENÉTICO
----------------------------------------------------------- """
exampleGen = Genetico(_genes = [0, 1],
             _longitud = n,
             _tamanoPoblacion = 17,
             _probCruce = 0.9,
             _probMutacion = 0.1,
             _puntoCruce = 4)

#exampleGen.show()
exampleGen.run()
#exampleGen.show()

>> Algoritmo inicialiado.
>> Ejecutando iteraciones...
••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••••
>> Ejecución terminada.
>> Factibilidad: True
>> Mejor solución: [1, 0, 0, 0, 0, 1, 0, 1, 0] Fitness: 17
